## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


In [2]:
from tensorflow.keras.layers import ReLU,Dense, BatchNormalization, Softmax, GlobalAveragePooling2D

In [3]:
import tensorflow_hub as hub

## Hyper Parameters

In [4]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100
img_size = 224

## Cifar10 Data

In [5]:
## Cifar10 Dataset #########################################################
cifar = keras.datasets.cifar10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
############################################################################

## Datasets

In [6]:
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()    
    
train_images = train_images.astype(np.float32)/255.
test_images = test_images.astype(np.float32)/255.
    
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## Loading Pretrained Model

In [7]:
## tf_hub에 있는 pretrained model url
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [8]:
## hub.KerasLayer를 이용하여 tf_hub에서 pretrained model loading
feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=(img_size, img_size, 3))

In [19]:
## train_dataset에서 1개의 batch만 가져와서 pretrained model에 넣고 output shape을 출력
for images, labels in train_dataset.take(1):
    images = tf.image.resize(images, (img_size, img_size))
    feature_maps = feature_extractor_layer(images)
    print(feature_maps.shape)

(100, 1280)


## Model Function

In [9]:
def create_model():
    model = keras.models.Sequential()
    model.add(feature_extractor_layer)
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Softmax())
    return model

In [10]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
softmax (Softmax)            (None, 10)                0

## Loss Function

In [14]:
@tf.function
def loss_fn(model, images, labels):
    predictions = model(images, training=True)
    loss = tf.reduce_mean(keras.losses.categorical_crossentropy(labels, predictions))   
    return loss   

## Calculating Gradient

In [15]:
@tf.function
def train(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

## Caculating Model's Accuracy

In [16]:
@tf.function
def evaluate(model, images, labels):
    predictions = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    
    return accuracy

## Optimizer

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

## Training

In [18]:
## train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        images = tf.image.resize(images, (img_size, img_size)) 
        train(model,images, labels)
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:
        images = tf.image.resize(images, (img_size, img_size))
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.69330543 train accuracy =  0.8216 test accuracy =  0.8416
Epoch: 2 loss = 0.47141683 train accuracy =  0.8741 test accuracy =  0.8522
Epoch: 3 loss = 0.37201279 train accuracy =  0.8932 test accuracy =  0.8531
Epoch: 4 loss = 0.29823610 train accuracy =  0.9114 test accuracy =  0.8518
Epoch: 5 loss = 0.23590051 train accuracy =  0.9267 test accuracy =  0.8494
Epoch: 6 loss = 0.18205740 train accuracy =  0.9401 test accuracy =  0.8454
Epoch: 7 loss = 0.13634875 train accuracy =  0.9533 test accuracy =  0.8448
Epoch: 8 loss = 0.09920695 train accuracy =  0.9644 test accuracy =  0.8409
Epoch: 9 loss = 0.07017039 train accuracy =  0.9738 test accuracy =  0.8409
Epoch: 10 loss = 0.04906841 train accuracy =  0.9819 test accuracy =  0.8381
Epoch: 11 loss = 0.03448673 train accuracy =  0.9873 test accuracy =  0.8404
Epoch: 12 loss = 0.02452154 train accuracy =  0.9911 test accuracy =  0.8442
Epoch: 13 loss = 0.01796298 train accuracy =  0.